# Regression Week 5: Feature Selection and LASSO (Interpretation)

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using GraphLab Create, though you can use other solvers). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

# Fire up graphlab create

In [1]:
import graphlab

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
sales = graphlab.SFrame('kc_house_data.gl/')

This non-commercial license of GraphLab Create for academic use is assigned to 15bit116@ietdavv.edu.in and will expire on September 10, 2017.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: C:\Users\chinesh\AppData\Local\Temp\graphlab_server_1475930300.log.0


# Create new features

As in Week 2, we consider features that are some transformations of inputs.

In [3]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].astype(float) 
sales['floors_square'] = sales['floors']*sales['floors']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [4]:
all_features = ['bedrooms', 'bedrooms_square',
            'bathrooms',
            'sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']

Applying L1 penalty requires adding an extra parameter (`l1_penalty`) to the linear regression call in GraphLab Create. (Other tools may have separate implementations of LASSO.)  Note that it's important to set `l2_penalty=0` to ensure we don't introduce an additional L2 penalty.

In [5]:
model_all = graphlab.linear_regression.create(sales, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=1e10)

Linear regression:

--------------------------------------------------------

Number of examples          : 21613

Number of features          : 17

Number of unpacked features : 17

...

| 5         | 6        | 0.000002  | 1.803399     | 6869667.895833     | 385998.458623 |

| 6         | 7        | 0.000002  | 1.906749     | 6847177.773672     | 380824.455891 |

+-----------+----------+-----------+--------------+--------------------+---------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

Find what features had non-zero weight.

In [9]:
model_all.get("coefficients").print_rows(num_rows=len(all_features)+1)

+------------------+-------+---------------+--------+
|       name       | index |     value     | stderr |
+------------------+-------+---------------+--------+
|   (intercept)    |  None |  274873.05595 |  None  |
|     bedrooms     |  None |      0.0      |  None  |
| bedrooms_square  |  None |      0.0      |  None  |
|    bathrooms     |  None | 8468.53108691 |  None  |
|   sqft_living    |  None | 24.4207209824 |  None  |
| sqft_living_sqrt |  None | 350.060553386 |  None  |
|     sqft_lot     |  None |      0.0      |  None  |
|  sqft_lot_sqrt   |  None |      0.0      |  None  |
|      floors      |  None |      0.0      |  None  |
|  floors_square   |  None |      0.0      |  None  |
|    waterfront    |  None |      0.0      |  None  |
|       view       |  None |      0.0      |  None  |
|    condition     |  None |      0.0      |  None  |
|      grade       |  None | 842.068034898 |  None  |
|    sqft_above    |  None | 20.0247224171 |  None  |
|  sqft_basement   |  None |

Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

***QUIZ QUESTION***:
According to this list of weights, which of the features have been chosen? 

# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

In [27]:
(training_and_validation, testing) = sales.random_split(.9,seed=1) # initial train/test split
(training, validation) = training_and_validation.random_split(0.5, seed=1) # split training into train and validate
list_l1 = np.logspace(1,7,num=13)
print list_l1

[  1.00000000e+01   3.16227766e+01   1.00000000e+02   3.16227766e+02
   1.00000000e+03   3.16227766e+03   1.00000000e+04   3.16227766e+04
   1.00000000e+05   3.16227766e+05   1.00000000e+06   3.16227766e+06
   1.00000000e+07]


Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

In [23]:
import numpy as np
sort_table = []
list_l1 = np.logspace(1,7,num=13)
for l1_penalty in list_l1:
    model = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=l1_penalty,verbose=False)
    predictions = model.predict(validation)
    RSS = sum([(predictions[i]-validation[i]['price'])**2 for i in range(len(predictions))])
    print l1_penalty,RSS
    sort_table.append((RSS,l1_penalty))
print sort_table

10.0 6.25766285142e+14
31.6227766017 6.25766285362e+14
100.0 6.25766286058e+14
316.227766017 6.25766288257e+14
1000.0 6.25766295212e+14
3162.27766017 6.25766317206e+14
10000.0 6.25766386761e+14
31622.7766017 6.25766606749e+14
100000.0 6.25767302792e+14
316227.766017 6.25769507644e+14
1000000.0 6.25776517727e+14
3162277.66017 6.25799062845e+14
10000000.0 6.25883719085e+14
[(625766285142461.4, 10.0), (625766285362395.4, 31.622776601683793), (625766286057886.9, 100.0), (625766288257224.9, 316.2277660168379), (625766295212185.9, 1000.0), (625766317206077.8, 3162.2776601683795), (625766386760661.5, 10000.0), (625766606749281.4, 31622.776601683792), (625767302791633.2, 100000.0), (625769507643885.0, 316227.76601683791), (625776517727025.8, 1000000.0), (625799062845466.9, 3162277.6601683791), (625883719085424.5, 10000000.0)]


*** QUIZ QUESTIONS ***
1. What was the best value for the `l1_penalty`?
2. What is the RSS on TEST data of the model with the best `l1_penalty`?

In [30]:
model_all = graphlab.linear_regression.create(sales, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=3162277.66017)

Linear regression:

--------------------------------------------------------

Number of examples          : 21613

Number of features          : 17

Number of unpacked features : 17

...

| 5         | 6        | 0.000002  | 0.498519     | 5757505.912307     | 280323.122592 |

| 6         | 7        | 0.000002  | 0.526517     | 5678798.382509     | 276617.876204 |

+-----------+----------+-----------+--------------+--------------------+---------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

***QUIZ QUESTION***
Also, using this value of L1 penalty, how many nonzero weights do you have?

In [31]:
model_all.get("coefficients").print_rows(num_rows=len(all_features)+1)

+------------------+-------+------------------+--------+
|       name       | index |      value       | stderr |
+------------------+-------+------------------+--------+
|   (intercept)    |  None |  20545.0067891   |  None  |
|     bedrooms     |  None |  8153.17009784   |  None  |
| bedrooms_square  |  None |   1478.6210699   |  None  |
|    bathrooms     |  None |   24573.850595   |  None  |
|   sqft_living    |  None |  37.4901930253   |  None  |
| sqft_living_sqrt |  None |  1109.29426391   |  None  |
|     sqft_lot     |  None | -0.0174672640994 |  None  |
|  sqft_lot_sqrt   |  None |  149.511017471   |  None  |
|      floors      |  None |  20976.3308388   |  None  |
|  floors_square   |  None |  12271.7556599   |  None  |
|    waterfront    |  None |   581462.11884   |  None  |
|       view       |  None |  92954.9870623   |  None  |
|    condition     |  None |  6920.74310572   |  None  |
|      grade       |  None |  6204.68625715   |  None  |
|    sqft_above    |  None |   

In [32]:
predictions = model_all.predict(testing)
RSS = sum([(predictions[i] - testing[i]['price']) ** 2 for i in range(len(predictions))])
print RSS

1.58065003928e+14


# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [33]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [34]:
l1_penalty_values = np.logspace(8, 10, num=20)

Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model['coefficients']['value']` gives you an SArray with the parameters you learned.  If you call the method `.nnz()` on it, you will find the number of non-zero parameters!* 

In [36]:
model_all['coefficients']['value'].nnz()

info = []
for l1_penalty in l1_penalty_values:
    model = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0, l1_penalty=l1_penalty, verbose=False)
    nnz = model['coefficients']['value'].nnz()
    info.append((l1_penalty, nnz))
    

In [37]:
for x in enumerate(info):
    print x

(0, (100000000.0, 18))
(1, (127427498.57031322, 18))
(2, (162377673.91887242, 18))
(3, (206913808.11147901, 18))
(4, (263665089.87303555, 17))
(5, (335981828.62837881, 17))
(6, (428133239.8719396, 17))
(7, (545559478.11685145, 17))
(8, (695192796.17755914, 17))
(9, (885866790.41008317, 16))
(10, (1128837891.6846883, 15))
(11, (1438449888.2876658, 15))
(12, (1832980710.8324375, 13))
(13, (2335721469.0901213, 12))
(14, (2976351441.6313128, 10))
(15, (3792690190.7322536, 6))
(16, (4832930238.5717525, 5))
(17, (6158482110.6602545, 3))
(18, (7847599703.5146227, 1))
(19, (10000000000.0, 1))


Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzero` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzero` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

In [38]:
l1_penalty_min = l1_penalty_values[14]
l1_penalty_max = l1_penalty_values[15]
print l1_penalty_min, l1_penalty_max

2976351441.63 3792690190.73


***QUIZ QUESTIONS***

What values did you find for `l1_penalty_min` and`l1_penalty_max`? 

## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [39]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)

* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzero`.

In [40]:
sort_table = []

for l1_penalty in l1_penalty_values:
    model = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0, l1_penalty=l1_penalty, verbose=False)
    nnz = model['coefficients']['value'].nnz()
    if not nnz == max_nonzeros:
        continue
    predictions = model.predict(validation)
    RSS = sum([(predictions[i] - validation[i]['price']) ** 2 for i in range(len(predictions))])
    print l1_penalty, RSS
    sort_table.append((RSS, l1_penalty))
    
print sorted(sort_table)[0]

3448968612.16 1.04693748875e+15
3491933809.48 1.05114762561e+15
3534899006.81 1.05599273534e+15
3577864204.13 1.06079953176e+15
(1046937488751713.4, 3448968612.1634364)


***QUIZ QUESTIONS***
1. What value of `l1_penalty` in our narrow range has the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`?
2. What features in this model have non-zero coefficients?

In [41]:
best_model = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0, l1_penalty=3320073020.20013, verbose=False)
best_model.get('coefficients').print_rows(num_rows=len(all_features) + 1)

+------------------+-------+---------------+--------+
|       name       | index |     value     | stderr |
+------------------+-------+---------------+--------+
|   (intercept)    |  None | 215235.603644 |  None  |
|     bedrooms     |  None | 1108.36955956 |  None  |
| bedrooms_square  |  None |      0.0      |  None  |
|    bathrooms     |  None | 16496.2360732 |  None  |
|   sqft_living    |  None | 32.9384118477 |  None  |
| sqft_living_sqrt |  None | 719.868441786 |  None  |
|     sqft_lot     |  None |      0.0      |  None  |
|  sqft_lot_sqrt   |  None |      0.0      |  None  |
|      floors      |  None | 829.560064725 |  None  |
|  floors_square   |  None |      0.0      |  None  |
|    waterfront    |  None |      0.0      |  None  |
|       view       |  None |      0.0      |  None  |
|    condition     |  None |      0.0      |  None  |
|      grade       |  None | 3100.35992021 |  None  |
|    sqft_above    |  None | 30.8186652907 |  None  |
|  sqft_basement   |  None |